In [9]:
%load_ext autoreload
%autoreload 2

# Second cell
from zoom_info_module import zoominfo

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import pandas as pd
from tqdm import tqdm

In [11]:
session = zoominfo()

In [12]:
df = pd.read_excel('Validation Records for tracer - Individual - zoominfo company search.xlsx')

In [13]:
df = df[['first_name', 'last_name', 'Linkedin Company']]
df['search key'] = df.apply(lambda x: (x['first_name'], x['last_name'], x['Linkedin Company']), axis=1)

In [14]:
search_list = []
consecutive_errors = 0
log_file = "error_log_raffie.txt"

# Open the log file in append mode at the beginning to keep previous logs
with open(log_file, "a") as file:
    file.write("Error Log\n")
    file.write("=========\n")

# Using leave=False to prevent tqdm from leaving progress bars on new lines
for search_key in tqdm(df['search key'], leave=False):
    try:
        search_df = session.search(search_key[0],search_key[1],search_key[2])
        search_list.append(search_df)
        consecutive_errors = 0  # Reset the error counter on successful processing
    except Exception as e:
        consecutive_errors += 1
        error_message = f"Error processing {search_key}: {e}\n"
        # print(error_message)

        # Append the error message to the log file
        with open(log_file, "a") as file:
            file.write(error_message)
        
        if consecutive_errors >= 5:
            raise Exception("Five consecutive errors encountered. Stopping execution.")

# Reset the consecutive_errors counter if needed for further processing
consecutive_errors = 0

  0%|          | 7/7651 [02:55<42:59:24, 20.25s/it]